In [2]:
# Import Libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import os

/Users/h6x/anaconda3/envs/TDA/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# functions to get the file names in a directory - gdp files without .gdp extension
def get_file_names(directory):
    files = os.listdir(directory)
    return [f.split('.')[0] for f in files if f.endswith('.gdp')]

In [4]:
county_names = get_file_names('/Users/h6x/ORNL/git/GNN-experiment/generating_graphs/generated_graphs/simplices')

In [5]:
for county_name in county_names:
    print(county_name)

    #load the gdp file
    gdf = gpd.read_file(f'/Users/h6x/ORNL/git/GNN-experiment/generating_graphs/generated_graphs/simplices/{county_name}.gdp')

    #load the array from npx file
    arr = np.load(f'/Users/h6x/ORNL/git/GNN-experiment/generating_graphs/generated_graphs/simplices/{county_name}.npz')




    break

01089


In [6]:
gdf

,STCNTY,FIPS,EP_DISABL,sortedID,geometry
0,01089,01089010901,0.077428,0,"POLYGON ((-86.526 34.700, -86.520 34.701, -86...."
1,01089,01089010621,0.081365,1,"POLYGON ((-86.714 34.787, -86.714 34.801, -86...."
2,01089,01089011011,0.091864,2,"POLYGON ((-86.786 34.736, -86.786 34.744, -86...."
3,01089,01089001401,0.097113,3,"POLYGON ((-86.715 34.742, -86.715 34.746, -86...."
4,01089,01089011100,0.099738,4,"POLYGON ((-86.723 34.598, -86.723 34.624, -86...."
...,...,...,...,...,...
68,01089,01089002400,0.291339,68,"POLYGON ((-86.628 34.692, -86.627 34.698, -86...."
69,01089,01089011400,0.292651,69,"POLYGON ((-86.452 34.486, -86.451 34.487, -86...."
70,01089,01089000702,0.308399,70,"POLYGON ((-86.613 34.760, -86.611 34.760, -86...."
71,01089,01089003000,0.317585,71,"POLYGON ((-86.601 34.744, -86.600 34.747, -86...."


In [7]:
#get the array
arr

In [8]:
for key in arr.files:
    array = arr[key]
    print(f"Array under key '{key}':\n{array}")

Array under key 'arr_0':
[0]
Array under key 'arr_1':
[1]
Array under key 'arr_2':
[2]
Array under key 'arr_3':
[3]
Array under key 'arr_4':
[4]
Array under key 'arr_5':
[5]
Array under key 'arr_6':
[1 5]
Array under key 'arr_7':
[6]
Array under key 'arr_8':
[2 6]
Array under key 'arr_9':
[7]
Array under key 'arr_10':
[8]
Array under key 'arr_11':
[9]
Array under key 'arr_12':
[1 9]
Array under key 'arr_13':
[5 9]
Array under key 'arr_14':
[1 5 9]
Array under key 'arr_15':
[10]
Array under key 'arr_16':
[ 4 10]
Array under key 'arr_17':
[11]
Array under key 'arr_18':
[ 5 11]
Array under key 'arr_19':
[ 9 11]
Array under key 'arr_20':
[ 5  9 11]
Array under key 'arr_21':
[12]
Array under key 'arr_22':
[ 1 12]
Array under key 'arr_23':
[ 3 12]
Array under key 'arr_24':
[ 9 12]
Array under key 'arr_25':
[ 1  9 12]
Array under key 'arr_26':
[13]
Array under key 'arr_27':
[ 2 13]
Array under key 'arr_28':
[ 3 13]
Array under key 'arr_29':
[ 6 13]
Array under key 'arr_30':
[ 2  6 13]
Array u

Need to construct edge features, node features, edge index, label

In [10]:
import torch
from torch_geometric.data import Data

In [11]:
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)

In [12]:
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

In [13]:
data = Data(x=x, edge_index=edge_index)

In [14]:
data

Data(x=[3, 1], edge_index=[2, 4])

In [15]:
import pandas as pd
import torch
import torch_geometric
from torch_geometric.data import Dataset
import numpy as np 
import os
from tqdm import tqdm
# import deepchem as dc
# from rdkit import Chem 

print(f"Torch version: {torch.__version__}")
print(f"Cuda available: {torch.cuda.is_available()}")
print(f"Torch geometric version: {torch_geometric.__version__}")



Torch version: 2.0.1
Cuda available: False
Torch geometric version: 2.5.3


In [16]:
class MoleculeDataset(Dataset):
    def __init__(self, root, filename, test=False, transform=None, pre_transform=None):
        """
        root = Where the dataset should be stored. This folder is split
        into raw_dir (downloaded dataset) and processed_dir (processed data). 
        """
        self.test = test
        self.filename = filename
        super(MoleculeDataset, self).__init__(root, transform, pre_transform)
        
    @property
    def raw_file_names(self):
        """ If this file exists in raw_dir, the download is not triggered.
            (The download func. is not implemented here)  
        """
        return self.filename

    @property
    def processed_file_names(self):
        """ If these files are found in raw_dir, processing is skipped"""
        self.data = pd.read_csv(self.raw_paths[0]).reset_index()

        if self.test:
            return [f'data_test_{i}.pt' for i in list(self.data.index)]
        else:
            return [f'data_{i}.pt' for i in list(self.data.index)]
        

    def download(self):
        pass

    def process(self):
        self.data = pd.read_csv(self.raw_paths[0]).reset_index()
        featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
        for index, row in tqdm(self.data.iterrows(), total=self.data.shape[0]):
            # Featurize molecule
            mol = Chem.MolFromSmiles(row["smiles"])
            f = featurizer._featurize(mol)
            data = f.to_pyg_graph()
            data.y = self._get_label(row["HIV_active"])
            data.smiles = row["smiles"]
            if self.test:
                torch.save(data, 
                    os.path.join(self.processed_dir, 
                                 f'data_test_{index}.pt'))
            else:
                torch.save(data, 
                    os.path.join(self.processed_dir, 
                                 f'data_{index}.pt'))
            

    def _get_label(self, label):
        label = np.asarray([label])
        return torch.tensor(label, dtype=torch.int64)

    def len(self):
        return self.data.shape[0]

    def get(self, idx):
        """ - Equivalent to __getitem__ in pytorch
            - Is not needed for PyG's InMemoryDataset
        """
        if self.test:
            data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_test_{idx}.pt'))
        else:
            data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_{idx}.pt'))        
        return data

In [ ]:
#test